In [1]:
import argparse
from Bio import Entrez
import pandas as pd
from datetime import date
from datetime import datetime
import re
import os
import glob

## Set the Output Path

In [2]:
# determining the current year, month and day
today = date.today()
date_str = today.strftime("%Y_%m_%d")

# determining current hour and minute
now = datetime.now()
time_str = now.strftime("%H_%M")

# setting the output filename
output = "GEO_Compare.{}_{}".format(date_str, time_str)

In [3]:
output

'GEO_Compare.2022_03_08_21_54'

## Find the Second Newest Queried Excel Sheet (Old) and the Newest (New)

In [4]:
# get a list of filenames containing'Query'
cwd = os.getcwd()
Query_files = []
os.chdir(cwd)
for file in glob.glob("*.xlsx"):
    if 'Query'.casefold() in file.casefold():
        Query_files.append(file)

# sort the two lists in descending order
Query_files.sort(key=str.lower, reverse=True)

old_file = cwd + "/" + Query_files[1]
new_file = cwd + "/" + Query_files[0]

In [5]:
# get the newest HiChIP_Databases Excel and newest Query Excel
old_df = pd.read_excel(old_file)
new_df = pd.read_excel(new_file)

##  Compare Old and New GEO IDs

In [6]:
# make a regex to extract GEO ID
GEO_pattern = re.compile('GSE[0123456789]+')

# make a set of old GEO IDs
old_GEOs = set()
for x in old_df['GEO / Data link'].tolist():
    GEO = GEO_pattern.findall(x)
    old_GEOs.update(GEO)

# make a set of new GEO IDs
new_GEOs = set()
for x in new_df['GEO / Data link'].tolist():
    GEO = GEO_pattern.findall(x)
    new_GEOs.update(GEO)

# compare the sets of GEO IDs
differences_GEOs = new_GEOs.difference(old_GEOs)

In [7]:
# create an indicate column of papers whose GEO ID is not in the old table 
bools = []
for x in new_df['GEO / Data link'].tolist():
    if x in differences_GEOs:
        bools.append(True)
    else:
        bools.append(False)
differences_df = new_df.loc[bools]

In [8]:
# replace na with empty string
differences_df.fillna("", inplace=True)

/Users/Cardiff/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [9]:
differences_df

,Paper Title,DOI,Journal,Authors,Year,GEO / Data link,Any other information,Organism,Tissue/Cell Line,Potential HiChIP,Other matched data,Presenter,Date Added


## Save the Output File

In [10]:
differences_df.to_excel(output+".xlsx", index=False)